# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


## Generate Cities List

In [ ]:
#________________starer code_____________________________________#
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500) #changed to 15 from 1500 for testing
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500) #changed to 15 from 1500 for testing
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
#_________________________end starter code______________________________________#


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
#set up api/ create URL
import json

#target each citi in list "cities" found above
url = "http://api.openweathermap.org/data/2.5/weather?"

#build query url and exception
record_number = 0
count = 1
batch = 1

#define lists to capture from Open Weather Site
city_name = []
city_lat = []
city_long = []
temp = []
humidity = []
country = []
cloudiness = []
windspeed = []
date = []

#loop to create list of found data and grab info from api for dataframe
for i, x  in enumerate(cities):
    #set groups of 50
    if count <= 50:
        count +=1        
    else:
        batch += 1 #set of first 50
        count = 0       
    record_number += 1
    try:
        query_url = url + "appid=" + weather_api_key + "&q=" + x
        print(f'Processing Record {record_number} of set {batch} for City: {x}')
        cities_response = requests.get(query_url).json()
        
        #store data to lists
        city_name.append(cities_response['name'])
        city_lat.append(cities_response['coord']['lat'])
        city_long.append(cities_response['coord']['lon'])
        cloudiness.append(cities_response['clouds']['all'])
        temp.append(cities_response['main']['temp'])
        humidity.append(cities_response['main']['humidity']) 
        country.append(cities_response['sys']['country'])
        windspeed.append(cities_response['wind']['speed'])
        date.append(cities_response['dt'])
    except KeyError:
        print(f'No information found for city: {x}') 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
cities_dict = {"City Name" : city_name, "Latitude":city_lat, "Longitude":city_long, "Temperature": temp, "Humidity": humidity, "Cloudiness":cloudiness, "Windspeed":windspeed, "Date":date }
cities_df = pd.DataFrame(cities_dict)
cities_df['Date']=pd.to_datetime(cities_df['Date'], unit='s')
cities_df.to_csv("../output_data/Citiesweather.csv", index = False, header = True)

In [ ]:
cities_df

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#check for cities with high humidity
if len(cities_df.loc[cities_df['Humidity'] > 100]):
    #keep those with humidity less than 100
    high_humidity_df = cities_df.loc[cities_df['Humidity'] > 100]
    clean_cities_df = cities_df.loc[cities_df['Humidity'] <= 100]
    #Get the indices of cities that have humidity over 100%.
    high_humidity_label = cities_df[cities_df['Humidity']>100].index.tolist()
    print(high_humidity_label)
    print(high_humidity_df)
else:
    clean_cities_df = cities_df

In [ ]:
    ##__________________review with tutor_________________________________-##


    # Make a new DataFrame equal to the city data to drop all humidity outliers by index.
    #condition= cities_df.loc[cities_df['Humidity'] < 100]
    #clean_cities_df = cities_df.where(cond=cities_df.loc[cities_df['Humidity'] < 100],inplace=False)
    # Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


    #print(clean_city_data_df)
    #print(clean_cities_df)

In [ ]:
    # Extract relevant fields from the data frame
    # Export the City_Data into a csv
    clean_cities_df.to_csv("../output_data/clean_cities_data.csv", index = False, header = True)
    clean_cities_df['just_date'] = clean_cities_df['Date'].dt.date
    clean_cities_df

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
clean_cities_df.plot.scatter("Latitude", "Temperature")
plt.title(f'Latitude vs Temperature on {clean_cities_df["just_date"].unique()}')
plt.savefig("../output_data/LatvsTemp.png")
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
clean_cities_df.plot.scatter("Latitude", "Humidity")
plt.title("Latitude vs Humidity")
plt.savefig("../output_data/LatvsHumid.png")
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
clean_cities_df.plot.scatter("Latitude", "Cloudiness")
plt.title("Latitude vs Humidity")
plt.savefig("../output_data/LatvsClouds.png")
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
clean_cities_df.plot.scatter("Latitude", "Windspeed")
plt.title("Latitude vs Windspeed")
plt.savefig("../output_data/LatvsWS.png")
plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

def linearregression(x,y,xlabel, ylabel, annotate):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
    regress_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x,y)
    plt.plot(x,regress_values,"r-")
    plt.annotate(line_eq,annotate,fontsize=15,color="red")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(f'{xlabel} "vs" {ylabel}')
    plt.show()
    


In [ ]:
# Create Northern and Southern Hemisphere DataFrames
northern_hemisphere_df = clean_cities_df.loc[clean_cities_df['Latitude']>=0]
northern_hemisphere_df.dropna(how = 'all', axis =1)
southern_hemisphere_df = clean_cities_df.loc[clean_cities_df['Latitude']<0]
southern_hemisphere_df.dropna(how = 'all', axis =1)

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#call function
linearregression(northern_hemisphere_df['Temperature'],northern_hemisphere_df['Latitude'], "Max Temperature", "Latitude", (280,20))
#sav fig
plt.savefig("../output_data/NorthMaxTempvsLat.png")
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#call function
linearregression(southern_hemisphere_df['Temperature'],southern_hemisphere_df['Latitude'], "Max Temperature", "Latitude", (290,-50))
#save figure
plt.savefig("../output_data/SouthMaxTempvsLat.png")
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Call function
linearregression(northern_hemisphere_df['Humidity'],northern_hemisphere_df['Latitude'], "Humidity", "Latitude", (20,20))
#save figure
plt.savefig("../output_data/NorthHumidvsLat.png")
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#call function
linearregression(southern_hemisphere_df['Humidity'],southern_hemisphere_df['Latitude'], "Humidity", "Latitude", (50, -40))
#save figure
plt.savefig("../output_data/SouthHumidvsLat.png")
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#call function
linearregression(northern_hemisphere_df['Cloudiness'],northern_hemisphere_df['Latitude'], "Cloudiness", "Latitude", (50,50))
#Sav fig
plt.savefig("../output_data/NorthCloudvsLat.png")
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#call function
linearregression(southern_hemisphere_df['Cloudiness'],southern_hemisphere_df['Latitude'], "Cloudiness", "Latitude", (50,-50))
#sav fig
plt.savefig("../output_data/SouthCloudvsLat.png")
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#call function
linearregression(northern_hemisphere_df['Windspeed'],northern_hemisphere_df['Latitude'],"Windspeed", "Latitude", (2,50) )
#sav fig
plt.savefig("../output_data/NorthWSvsLat.png")
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Call function
linearregression(southern_hemisphere_df['Windspeed'],southern_hemisphere_df['Latitude'], "Windspeed", "Latitude", (6, -60))
#sav fig
plt.savefig("../output_data/SouthWSvsLat.png")
plt.show()